# Задача
Необходимо создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. Для оценки использовать метрику MAPE

В данном ноутбуке выбрана модель для предсказания.

В этом проекте использовали:
Ноутбук, через который парсили https://www.kaggle.com/igorchernov/sf-dst-car-price-parser

Спарсенный датасет https://www.kaggle.com/igorchernov/input/data-car-sales/

Ноутбук, в котором провели EDA https://www.kaggle.com/igorchernov/sf-dst-car-price-eda

Ноутбук, в котором провели обучение https://www.kaggle.com/igorchernov/sf-dst-car-price-ml



# Загрузка данных

In [26]:
# Импорт библиотек

import os
import numpy as np
import pandas as pd
from pandas import Series

import re
import datetime

from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.model_selection import train_test_split, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from tqdm.notebook import tqdm
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor

In [27]:
import warnings 
warnings.filterwarnings("ignore") 

In [28]:
RANDOM_SEED = 42

In [29]:
#Импорт данных из соревнования

data_sample = pd.read_csv(os.path.join(os.getcwd(),'input/sample_submission.csv'))

#Импорт обработанных данных
data_full = pd.read_csv(os.path.join(os.getcwd(),'input/data_car_eda.csv')) 

#Выбранные признаки в EDA
data_columns = pd.read_csv(os.path.join(os.getcwd(),'input/data_full_columns.csv')) 

cat_cols = data_columns[data_columns.column_type == 'cat'].column_name.values
bin_cols = data_columns[data_columns.column_type == 'bin'].column_name.values
num_cols = data_columns[data_columns.column_type == 'num'].column_name.values

print(len(data_full))

183246


Сначала выделим тестовую и тренировочную части.

In [30]:
data_train = data_full[data_full.sample_ == 0]
data_test = data_full[data_full.sample_ == 1].drop(['price', 'sample_'], axis=1).values

Делим данные на еще один тест и трейн, для валидации,
чтобы проверить, как хорошо модель работает, до отправки submission на kaggle.

In [31]:
X = data_train.drop(['price', 'sample_'], axis=1).values
Y = data_train['price'].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=RANDOM_SEED)

In [32]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

def learn_model(model):        
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)    
    print (f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
    
def learn_model_log(model):
    model.fit(X_train,np.log(y_train+1))
    y_pred = np.exp(model.predict(X_test))
    print (f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Далее попробуем построить разные модели и посмотрим их метрики. Все модели закомментируем, чтобы не грузить ноутбук. В комментариях пропишем метрики. Оставим для загрузки только лучшую модель

# Наивная модель

In [33]:
#learn_model(LinearRegression())
#Точность модели по метрике MAPE: 87.83%

# Простые модели

In [34]:
# Построим линейную регрессию с логарифмированием целевой переменной

lr = LinearRegression().fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(lr.predict(X_test))))*100:0.2f}%")

predict_test = np.exp(lr.predict(X_test))
predict_submission = np.exp(lr.predict(data_test))

# Точность модели по метрике MAPE: 20.95%

Точность модели по метрике MAPE: 20.95%


In [35]:
# Random forest

rf = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1).fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf.predict(X_test))))*100:0.2f}%")

predict_test = np.exp(rf.predict(X_test))
predict_submission = np.exp(rf.predict(data_test))

# Точность модели по метрике MAPE: 14.29%

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   17.6s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s


Точность модели по метрике MAPE: 14.29%


[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [36]:
# Подбор параметров для Random forest

# random_grid = {'n_estimators': [int(x) for x in np.linspace(start=200, stop=2000, num=10)],
#                'max_features': ['auto', 'sqrt'],
#                'max_depth': [int(x) for x in np.linspace(10, 110, num=11)],
#                'min_samples_split': [2, 5, 10],
#                'min_samples_leaf': [1, 2, 4],
#                'bootstrap': [True, False]
#                }

# rf = RandomForestRegressor(random_state=RANDOM_SEED)
# rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100,
#                                cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1).fit(X_train, np.log(y_train+1))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rSCV.predict(X_test))))*100:0.2f}%")

In [37]:
# rf_random.best_params_

In [38]:
# rf = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1,
#                            max_features=3, max_depth=25, bootstrap=True, random_state=RANDOM_SEED).fit(X_train, np.log(y_train+1))

# predict_test = np.exp(rf.predict(X_test))
# predict_submission = np.exp(rf.predict(data_test))

# print(
#     f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# Точность модели по метрике MAPE: 16.21%

In [39]:
# еще парочка экспериментов

learn_model_log(KNeighborsRegressor(
    algorithm='ball_tree', weights='distance', p=1))
# Точность модели по метрике MAPE: 39.89%

learn_model_log(KNeighborsRegressor(
    algorithm='ball_tree', weights='distance', p=1))
# Точность модели по метрике MAPE: 39.89%

learn_model(DecisionTreeRegressor(random_state=RANDOM_SEED))
# Точность модели по метрике MAPE: 20.08%

dtr = DecisionTreeRegressor(max_depth=15, random_state=RANDOM_SEED)
learn_model_log(dtr)
# Точность модели по метрике MAPE: 16.41%

Точность модели по метрике MAPE: 39.89%
Точность модели по метрике MAPE: 39.89%
Точность модели по метрике MAPE: 20.08%
Точность модели по метрике MAPE: 16.41%


# Бустинг

In [40]:
# CatBoostRegressor

cb = CatBoostRegressor(iterations=5000, random_seed=RANDOM_SEED, eval_metric='MAPE',
                       custom_metric=['R2', 'MAE'], silent=True,)
cb.fit(X_train, np.log(y_train+1), eval_set=(X_test, np.log(y_test+1)),
       verbose_eval=0, use_best_model=True)

print(
    f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(cb.predict(X_test))))*100:0.2f}%")

predict_test = np.exp(cb.predict(X_test))
predict_submission = np.exp(cb.predict(data_test))

# Точность модели по метрике MAPE: 13.87%

Точность модели по метрике MAPE: 13.87%


In [41]:
# GradientBoostingRegressor

gb = GradientBoostingRegressor(
    min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)
gb.fit(X_train, np.log(y_train+1))

print(
    f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(gb.predict(X_test))))*100:0.2f}%")

predict_test = np.exp(gb.predict(X_test))
predict_submission = np.exp(gb.predict(data_test))

# Точность модели по метрике MAPE: 13.88%

Точность модели по метрике MAPE: 13.88%


In [42]:
# xgboost

xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03,
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))

print(
    f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(xb.predict(X_test))))*100:0.2f}%")

predict_test_final = np.exp(xb.predict(X_test))
predict_submission_final = np.exp(xb.predict(data_test))

# Точность модели по метрике MAPE: 13.75%

Точность модели по метрике MAPE: 13.75%


# Стекинг

In [43]:
def compute_meta_feature(rgr, X_train, X_test, y_train, cv, data_test):
    
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_regr = clone(rgr)
        folded_regr.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)

    meta_regr = clone(rgr)
    meta_regr.fit(X_train, y_train)

    X_meta_test = meta_regr.predict(X_test)
    X_meta_pred = meta_regr.predict(data_test)

    return X_meta_train, X_meta_test, X_meta_pred

In [44]:
def generate_meta_features(rgrs, X_train, X_test, y_train, cv, data_test):

    features = [compute_meta_feature(
        rgr, X_train, X_test, y_train, cv, data_test) for rgr in tqdm(rgrs)]

    stacked_features_train = np.vstack(
        [features_train for features_train, features_test, features_pred in features]).T
    stacked_features_test = np.vstack(
        [features_test for features_train, features_test, features_pred in features]).T
    stacked_features_pred = np.vstack(
        [features_pred for features_train, features_test, features_pred in features]).T

    return stacked_features_train, stacked_features_test, stacked_features_pred

In [45]:
def compute_metric(rgr, X_train, y_train, X_test, y_test):

    rgr.fit(X_train, np.log(y_train+1))
    y_test_pred = np.exp(rgr.predict(X_test))

    return np.round(mape(y_test, y_test_pred)*100, 4)

In [48]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
data_test = scaler.transform(data_test)

cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features([
    rf, gb, xb], X_train, X_test, y_train, cv, data_test)

  0%|          | 0/3 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.6s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   16.2s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6

In [49]:
print(
    f"Точность модели по метрике MAPE: {compute_metric(rf, stacked_features_train, y_train, stacked_features_test, y_test)}%")

predict_test = np.exp(rf.predict(stacked_features_test))
predict_submission = np.exp(rf.predict(stacked_features_pred))

# Точность модели по метрике MAPE: 14.582%

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    7.7s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s


Точность модели по метрике MAPE: 14.582%


[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.2s finished


## Выводы следующие:
1. Catboost достаточно хороший алгоритм, и работает даже с пропусками.
2. Превзойти результат Catboost удалось только с xgboost. Его и будем использовать в соревовании
3. Метрику сильно улучшает логарифмирование целевой переменной
4. Обучение больших датасетов (250к обьектов и 20+ признаков) требует хороших ресурсов. Не все можно запустить на кагле (например, глубина дерева, поиск гиперпараметров и подобное вылетают с ошибкой нехватки памяти...)

## Что бы мы сделали еще, но из-за ограниченности по времени, оставим в таком виде:
1. Изменить способ кодирования признаков
2. Попробовать использовать признак с моделями авто, который мы удалили
3. Прологарифмировать все числовые признаки
4. Сделать целевую переменную категориальной (в диапазоне +-2сигма с шагом 10к) и применить модели классификации
5. Попробовать стекинг лучших алгоритмов GradientBoostingRegressor, XGBRegressor и Catboost

# Submission

In [58]:
data_sample['price'] = predict_submission_final
data_sample.to_csv(os.path.join(os.getcwd(),'output/submission.csv'), index=False)
data_sample.head(10)

,sell_id,price
0,1100575026,6.508907e+05
1,1100549428,8.842190e+05
2,1100658222,9.356661e+05
3,1100937408,7.685855e+05
4,1101037972,8.155594e+05
5,1100912634,8.140340e+05
6,1101228730,7.141589e+05
7,1100165896,4.789025e+05
8,1100768262,2.018760e+06
9,1101218501,1.006762e+06


In [59]:
len(predict_submission_final)

34686